딥러닝 기반 Super-Resolution(SR) 분야의 초기 모델인 SRCNN과 잔차학습을 도입하여 깊은 네트워크 구조로 세밀한 특징을 학습하고 픽셀 수준에서 복원 성능을 향상시킨 VDSR 모델을 활용해 저화질 자동차 번호판 이미지를 개선하고자 한다.

SRCNN 모델은 저해상도(LR) 이미지를 입력으로 받아 고해상도(HR) 이미지로 변환하기 위해 3개의 컨볼루션 레이어를 사용한다.
첫 번째 컨볼루션 레이어는 9x9 필터를 적용하며, 입력 이미지의 크기를 유지하기 위해 패딩 4를 사용한다. 이 과정에서 이미지의 저차원 공간에 존재하는 중요한 특징을 추출하여 64개의 특징맵으로 변환한다. 이때 활성화 함수로 ReLU를 사용해 비선형성을 추가하고, 추출된 특징은 다음 레이어로 전달된다.

두 번째 컨볼루션 레이어에서는 5x5 필터를 사용하여 64개의 특징맵을 32개의 특징맵으로 축소하며, 이미지 크기를 유지하기 위해 패딩 2를 적용한다. 이 레이어는 ReLU 활성화 함수를 통해 저해상도 특징과 고해상도 특징 간의 매핑을 수행한다.

마지막 세 번째 컨볼루션 레이어는 5x5 필터와 패딩 2를 사용하여 32개의 특징맵을 최종적으로 3채널(RGB)의 고해상도 이미지로 변환한다. 이렇게 학습된 모델은 입력 이미지를 기반으로 고해상도 이미지를 재구성하여 최종 출력을 생성한다.


In [1]:
import tensorflow.compat.v1 as tf1
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
session = tf1.Session(config=config)

2024-12-22 04:14:37.891265: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-22 04:14:37.934674: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-22 04:14:38.633108: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-12-22 04:14:39.816153: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 36305 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:5e:00.0, compute capab

In [5]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm


# 평가지표
# PSNR 함수
def calculate_psnr(img1, img2, max_val=1.0):
    mse = F.mse_loss(img1, img2, reduction='mean')
    if mse == 0:
        return float('inf')
    return 10 * torch.log10(max_val**2 / mse)

# SSIM 함수
def calculate_ssim(img1, img2, max_val=1.0):
    C1 = (0.01 * max_val) ** 2
    C2 = (0.03 * max_val) ** 2

    mu1 = F.avg_pool2d(img1, kernel_size=3, stride=1, padding=1)
    mu2 = F.avg_pool2d(img2, kernel_size=3, stride=1, padding=1)

    sigma1_sq = F.avg_pool2d(img1 * img1, kernel_size=3, stride=1, padding=1) - mu1.pow(2)
    sigma2_sq = F.avg_pool2d(img2 * img2, kernel_size=3, stride=1, padding=1) - mu2.pow(2)
    sigma12 = F.avg_pool2d(img1 * img2, kernel_size=3, stride=1, padding=1) - mu1 * mu2

    ssim_map = ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) / ((mu1.pow(2) + mu2.pow(2) + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

# LR Image 전처리 단계임
def create_low_res_images(input_dir, output_dir, scale):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('png', 'jpg', 'jpeg')):
            img = Image.open(os.path.join(input_dir, filename))
            lr_img = img.resize((img.width // scale, img.height // scale), Image.Resampling.BICUBIC)
            lr_img.save(os.path.join(output_dir, filename))

def get_image_paths(directory):
    return sorted([os.path.join(directory, f) for f in os.listdir(directory) if f.lower().endswith(('png', 'jpg', 'jpeg'))])

class SRDataset(Dataset):
    def __init__(self, hr_dir, lr_dir, transform=None, input_size=(64, 64)):
        self.hr_paths = get_image_paths(hr_dir)
        self.lr_paths = get_image_paths(lr_dir)
        self.transform = transform
        self.input_size = input_size

    def __len__(self):
        return len(self.hr_paths)

    def __getitem__(self, idx):
        hr_img = Image.open(self.hr_paths[idx]).convert("RGB")
        lr_img = Image.open(self.lr_paths[idx]).convert("RGB")

        lr_img = lr_img.resize(self.input_size, Image.Resampling.BICUBIC)
        hr_img = hr_img.resize(self.input_size, Image.Resampling.BICUBIC)  # 타겟 크기 맞춤

        if self.transform:
            hr_img = self.transform(hr_img)
            lr_img = self.transform(lr_img)

        return lr_img, hr_img

# SRCNN 모델 정의
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x


def train_model(model, train_loader, valid_loader, criterion, optimizer, device, num_epochs, save_path, patience=5):
    best_valid_loss = float('inf')
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss, train_psnr = 0, 0

        for lr_imgs, hr_imgs in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{num_epochs}"):
            lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
            preds = model(lr_imgs)
            loss = criterion(preds, hr_imgs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_psnr += calculate_psnr(preds, hr_imgs).item()

        train_loss /= len(train_loader)
        train_psnr /= len(train_loader)

        model.eval()
        valid_loss, valid_psnr, valid_ssim = 0, 0, 0

        with torch.no_grad():
            for lr_imgs, hr_imgs in tqdm(valid_loader, desc="Validating"):
                lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
                preds = model(lr_imgs)
                valid_loss += criterion(preds, hr_imgs).item()
                valid_psnr += calculate_psnr(preds, hr_imgs).item()
                valid_ssim += calculate_ssim(preds, hr_imgs).item()

        valid_loss /= len(valid_loader)
        valid_psnr /= len(valid_loader)
        valid_ssim /= len(valid_loader)

        print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train PSNR: {train_psnr:.4f} | "
              f"Valid Loss: {valid_loss:.4f} | Valid PSNR: {valid_psnr:.4f} | Valid SSIM: {valid_ssim:.4f}")

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), save_path)
            print("Best model saved!")
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("Early stopping triggered!")
            break


def main():
    train_hr_dir = "/home/a202192006/image/train"
    train_lr_dir = "/home/a202192006/image/train_lr"
    valid_hr_dir = "/home/a202192006/image/valid"
    valid_lr_dir = "/home/a202192006/image/valid_lr"

    # LR Image 만듬
    create_low_res_images(train_hr_dir, train_lr_dir, scale=4)
    create_low_res_images(valid_hr_dir, valid_lr_dir, scale=4)

    transform = transforms.ToTensor()
    train_dataset = SRDataset(train_hr_dir, train_lr_dir, transform, input_size=(64, 64))
    valid_dataset = SRDataset(valid_hr_dir, valid_lr_dir, transform, input_size=(64, 64))

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = SRCNN().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    save_path = "/home/a202192006/image/srcnn/SRCNN_Earlystopping_best_model.pth"

    print("Training SRCNN...")
    train_model(model, train_loader, valid_loader, criterion, optimizer, device, num_epochs=100, save_path=save_path, patience=5)


    test_dataset = SRDataset(valid_hr_dir, valid_lr_dir, transform, input_size=(64, 64))
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    model.load_state_dict(torch.load(save_path))
    model.eval()

    test_psnr, test_ssim = 0, 0
    with torch.no_grad():
        for lr_imgs, hr_imgs in tqdm(test_loader, desc="Testing"):
            lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
            preds = model(lr_imgs)
            test_psnr += calculate_psnr(preds, hr_imgs).item()
            test_ssim += calculate_ssim(preds, hr_imgs).item()

    test_psnr /= len(test_loader)
    test_ssim /= len(test_loader)

    print(f"SRCNN - Test PSNR: {test_psnr:.4f}, Test SSIM: {test_ssim:.4f}")

if __name__ == "__main__":
    main()


Training SRCNN...


Training Epoch 1/100:   0%|                                                                                                                                    | 0/17500 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Validating:   0%|                                                                                                                                               | 0/1750 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Epoch 1/100 | Train Loss: 0.0044 | Train PSNR: 24.3243 | Valid Loss: 0.0038 | Valid PSNR: 24.8610 | Valid SSIM: 0.7685
Best model saved!


Training Epoch 2/100:   0%|                                                                                                                                    | 0/17500 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Validating:   0%|                                                                                                                                               | 0/1750 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Epoch 2/100 | Train Loss: 0.0038 | Train PSNR: 24.8316 | Valid Loss: 0.0037 | Valid PSNR: 25.0824 | Valid SSIM: 0.7768
Best model saved!


...

Epoch 58/100 | Train Loss: 0.0030 | Train PSNR: 25.8310 | Valid Loss: 0.0030 | Valid PSNR: 25.9834 | Valid SSIM: 0.8090


Training Epoch 59/100:   0%|                                                                                                                                   | 0/17500 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Validating:   0%|                                                                                                                                               | 0/1750 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Epoch 59/100 | Train Loss: 0.0030 | Train PSNR: 25.8375 | Valid Loss: 0.0030 | Valid PSNR: 25.9994 | Valid SSIM: 0.8093
Early stopping triggered!


Testing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7000/7000 [05:36<00:00, 20.80it/s]


SRCNN - Test PSNR: 27.3107, Test SSIM: 0.8090
